In [ ]:
!pip install python-binance

In [ ]:
from binance.client import Client
import pandas as pd
from websocket import create_connection
from datetime import datetime, timedelta

import csv
import os
import time
import datetime
import numpy as np
import dateparser as dp
from tqdm import tqdm
import psutil
import boto3

In [ ]:
client = Client(<binance_key>, <binance_secret>, {"verify": True, "timeout": 20})

client.ping()
time_res = client.get_server_time()
sysStatus = client.get_system_status()
exchinfo = client.get_exchange_info()
dfColumns = ['Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','Taker buy base asset volume','Taker buy quote asset volume','Ignore']

In [ ]:
btcPairedSymbols = [exchinfo.get('symbols')[i].get('symbol') for i in range(len(exchinfo.get('symbols'))) if exchinfo.get('symbols')[i].get('quoteAsset') == 'BTC']

len(btcPairedSymbols)

In [ ]:
dataLocationCommonPath = <path_to_download_dir_on_s3_bucket>
s3Url = <s3_URL>
BUCKET_NAME = <s3_bucket_name>

### Downloading data for BTC paired coins

In [ ]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(BUCKET_NAME)

downloadedException = True

fromdate = '1 Jan, 2018'
todate = '1 Jan, 2020'

fromdatetime = dp.parse(fromdate).strftime('%Y-%m-%d_%H%M%S')
todatetime = dp.parse(todate).strftime('%Y-%m-%d_%H%M%S')

while(downloadedException == True):
    try:
        downloadedException = False

        for sym in tqdm(btcPairedSymbols):
            alreadyDownloaded = False

            for my_bucket_object in my_bucket.objects.filter(Prefix=dataLocationCommonPath):
                if ('.csv' in my_bucket_object.key) and (my_bucket_object.key.split('/')[-1].split('_')[0] == sym):
                    alreadyDownloaded = True
                    break

            if alreadyDownloaded == True:
                continue

            print("Creating dataframe for: " + sym + "*"*50)
            print(psutil.virtual_memory())

            klines = client.get_historical_klines(sym, Client.KLINE_INTERVAL_1MINUTE, fromdate, todate)
            if (len(klines) > 0):
                df = pd.DataFrame(klines)
                df[0] = pd.to_datetime(df[0], unit='ms')
                df.set_index(0, inplace=True)
                df.columns = dfColumns
                df.to_csv(s3Url + sym + "_from_" + fromdatetime + "_To_" + todatetime + '.csv', index=True)
            else:
                print("Data not retrieved from symbol:" + sym)
    except:
        print("Symbol [%s] was skipped due to exeception" %sym)
        downloadedException = True
        time.sleep(10)